# Genetic algorithm: Sudoku V2

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import default_rng
import matplotlib.pyplot as plt
import csv
rng = default_rng(6502729091)

## Genotype definition
Array of 81 elements. Every 9 elements represent an inner square, starting from the top left to the bottom right.

In [ ]:
def SquareToLinear(arr):
    result = np.zeros(81, dtype=int)
    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            result[idx * 9 : (idx + 1) * 9] = np.reshape(arr[i * 3:(i + 1) * 3, j * 3 : (j + 1) * 3], (9))
    return result

In [ ]:
def LinearToSquare(arr):
    result = np.zeros((9,9), dtype=int)
    for i in range(3):
        for j in range(3):
            idx = i * 3 + j
            result[i * 3:(i + 1) * 3, j * 3:(j + 1) * 3] = np.reshape(arr[idx * 9 : (idx + 1) * 9], (3,3))
    return result

In [ ]:
def FillObvious(genotype):
    board = LinearToSquare(genotype)
    possible = np.array([[{1,2,3,4,5,6,7,8,9} for i in range(9)] for i in range(9)])
    
    def DiscardRow(r, x):
        for s in possible[r]:
            s.discard(x)
    def DiscardColumn(c, x):
        for s in possible[:,c]:
            s.discard(x)
    def DiscardInnerSquare(n, m, x):
        for r in possible[n * 3:(n + 1) * 3, m * 3:(m + 1) * 3]:
            for s in r:
                s.discard(x)
    
    for i in range(9):
        for j in range(9):
            if board[i,j] != 0:
                DiscardRow(i, board[i,j])
                DiscardColumn(j, board[i,j])
                DiscardInnerSquare(i // 3, j // 3, board[i,j])
    
    changed = 0
    for i in range(9):
        for j in range(9):
            if len(possible[i,j]) == 1 and board[i,j] == 0:
                board[i,j] = list(possible[i,j])[0]
                changed += 1
    
    print(f'Filled {changed} values')
    return SquareToLinear(board)

In [ ]:
def ObjectiveFunction(genotype):
    ans = 0
    # Rows
    raux = np.reshape(genotype, (9,9))
    for i in range(3):
        for j in range(3):
            ans += 9 - len(np.unique(raux[i * 3 : (i + 1) * 3, j * 3 : (j + 1) * 3]))
    
    # Columns
    caux = np.reshape(genotype, (9,3,3))
    for i in range(3):
        for j in range(3):
            ans += 9 - len(np.unique(caux[i::3][:,:,j]))
    
    return ans

In [ ]:
def SolutionPermutation(initialState):
    solution = np.zeros(81, dtype=int)
    for i in range(9):
        perm = rng.permutation(9) + 1
        for j, x in enumerate(initialState[i * 9 : (i + 1) * 9]):
            if x != 0:
                perm[np.where(perm == x)[0]] = perm[j]
                perm[j] = x
        solution[i * 9 : (i + 1) * 9] = perm.copy()
        
    return solution

In [ ]:
def ReadFromCSV(path):
    ans = np.zeros((9,9), dtype=int)
    with open(path) as f:
        csvf = csv.reader(f, delimiter=',')
        for i, r in enumerate(csvf):
            ans[i] = r
    
    return SquareToLinear(ans)

## Genetic Algorithm

In [ ]:
def ParentSelectionTournament(fitness, k=2):
    '''
        Input:  fitness -> Array of pop_size elements
                k -> size of tournament
        Output: Index of selected parent
    '''
    selection = rng.permutation(len(fitness))
    not_selection = selection[k:]
    raffle = fitness.copy()
    raffle[not_selection] = np.max(raffle) + 1
    return np.argmin(raffle)

In [ ]:
def CrossoverSimplePermutation(parent1, parent2):
    final_offspring = []
    cut = rng.integers(low=1, high=8)
    return np.append(parent1[:cut * 9], parent2[cut * 9:])

In [ ]:
def MutationSwap(genotype, initialState):
    mutated = genotype.copy()
    helper = np.reshape(initialState, (9,9))
    mutated = np.reshape(mutated, (9,9))
    
    r_idx, cnt_not_fixed = None, None
    while True:
        r_idx = rng.integers(low=0, high=9)
        cnt_not_fixed = np.count_nonzero(helper[r_idx] == 0)
        if cnt_not_fixed >= 2:
            break
    
    g = rng.integers(low=0, high=cnt_not_fixed, size=2)
    
    c_idx1 = 0
    for i,x in enumerate(helper[r_idx]): 
        if g[0] == 0 and x == 0:
            c_idx1 = i
            break
        if x == 0:
            g[0] -= 1
    
    c_idx2 = 0
    for i,x in enumerate(helper[r_idx]): 
        if g[1] == 0 and x == 0:
            c_idx2 = i
            break
        if x == 0:
            g[1] -= 1
    
    mutated[r_idx,c_idx1], mutated[r_idx,c_idx2] = mutated[r_idx,c_idx2], mutated[r_idx,c_idx1]
    return np.reshape(mutated, (81))

In [ ]:
def SudokuCrossover(population, fitness, Pr):
    new_population = np.zeros(population.shape, dtype=int)
    for i in range(len(population)): 
        if(rng.random() < Pr):
            parent1 = population[ParentSelectionTournament(fitness)]
            parent2 = population[ParentSelectionTournament(fitness)]
            new_population[i] = CrossoverSimplePermutation(parent1, parent2)
        else:
            new_population[i] = population[ParentSelectionTournament(fitness)]
    
    return new_population

In [ ]:
def SudokuMutation(population, initialMutation, Pm):
    new_population = np.zeros(population.shape, dtype=int)
    for i, genotype in enumerate(population):
        new_population[i] = MutationSwap(genotype, initialMutation) if rng.random() < Pm else genotype
    return new_population

In [ ]:
def SudokuGeneticAlgorithm(initialState, N=30, G=100, Pr=0.8, Pm=0.3):
    population = np.array([SolutionPermutation(initialState) for i in range(N)])
    fitness = np.array([ObjectiveFunction(genotype) for genotype in population])
    elite = population[np.argmin(fitness)].copy()
    elite_fx = np.min(fitness)
    
    current_g = 0
    elite_last_updated = 0
    while current_g < G and elite_fx != 0:
        if current_g % 100 == 0:
            print(f'Generation #{current_g}: fitness = {elite_fx}')
            print(f'min = {np.min(fitness)}, max = {np.max(fitness)}, mean = {np.mean(fitness)}, std = {np.std(fitness)}')
        
        '''
        if current_g - elite_last_updated >= 2000:
            print('-----Restarting population-----')
            population = np.array([SolutionPermutation(initialState) for i in range(N)])
            fitness = np.array([ObjectiveFunction(genotype) for genotype in population])
            max_idx = ParentSelectionTournament(fitness * -1)
            population[max_idx] = elite
            fitness[max_idx] = elite_fx
            elite_last_updated = current_g
        '''
            
        population = SudokuCrossover(population, fitness, Pr)
        population = SudokuMutation(population, initialState, Pm)
        fitness = np.array([ObjectiveFunction(genotype) for genotype in population])
        if(np.min(fitness) > elite_fx):
            max_idx = ParentSelectionTournament(fitness * -1)
            population[max_idx] = elite
            fitness[max_idx] = elite_fx
        else:
            if np.min(fitness) != elite_fx:
                elite_last_updated = current_g
            elite = population[np.argmin(fitness)].copy()
            elite_fx = np.min(fitness)
            
        current_g += 1
    
    return elite, elite_fx

In [ ]:
def SolveSudokuFromCSV(path):
    initialState = ReadFromCSV(path)
    initialState = FillObvious(initialState)
    elite, elite_fx = SudokuGeneticAlgorithm(initialState, N=80, G=100000, Pm=0.3)
    elite = LinearToSquare(elite)
    initialState = LinearToSquare(initialState)
    nrows, ncols = len(elite), len(elite)
    image = np.zeros(nrows*ncols)

    def highlight_cell(x,y, ax=None, **kwargs):
        rect = plt.Rectangle((x-.5, y-.5), 1,1, fill=False, **kwargs)
        ax = ax or plt.gca()
        ax.add_patch(rect)
        return rect

    def highlight_separator(x, y, **kwargs):
        wx, wy = 0, 10
        if x == 0:
            wx, wy = wy, wx
        rect = plt.Rectangle((x-.5, y-0.5), wx, wy, fill=False, **kwargs)
        ax = plt.gca()
        ax.add_patch(rect)
        return rect

    image = image.reshape((nrows, ncols))

    plt.matshow(image, cmap='afmhot')
    for i in range(nrows):
        for j in range(ncols):
            highlight_cell(i, j, color='white', linewidth=2)
    highlight_separator(3, 0, color='white', linewidth=6)
    highlight_separator(6, 0, color='white', linewidth=6)
    highlight_separator(0, 3, color='white', linewidth=6)
    highlight_separator(0, 6, color='white', linewidth=6)

    for i, r in enumerate(elite):
        for j, x in enumerate(r):
            color = 'pink' if initialState[i,j] != 0 else 'white'
            plt.text(j, i, x, color=color, horizontalalignment='center', verticalalignment='center', fontsize=14, fontweight='bold')

    plt.show()

In [ ]:
SolveSudokuFromCSV('hard1.csv')

In [ ]:
SolveSudokuFromCSV('super_easy.csv')

In [ ]:
SolveSudokuFromCSV('example1.csv')